notes: 

- cloud14_19 contains 2014 -- 2019

- cloud11_13 has 2011 --> 2013

- arscl1cloth has 2001 --> 2010

- arsclcbh1cloth also has 2001 --> 2010 but no cloud mask, likely left out of analysis



Gaoyun

In addition, we evaluate the variations of deep- (cloud top height (CTH) > 8 km), shallow- (CTH <3km), and
convective congestus (CTH between 3 km and 8 km) associated with APEs based on hourly precipitation and cloud
fraction as in Zhuang et al. (2017, Figure S2).



Yizhou


Thresholds for cloud fraction and 20 dBZ rain fraction are first determined manually (0.2 and 0.02) before making a classification for the samples.


Shallow and congestus convection day (SC)/nontransition case: 

(1) No cloud and rain fraction is above 6 km between 08 and 18 LST; 

(2) cloud or rain fraction below 6 km lasts for more than 1 h between 08 and 18 LST.

Locally formed daytime deep convection day (LD) / transition case: 

(1) Cloud or rain fraction above 8 km occurs at any time between 08 and 18 LST, and the cloud base is below 2 km; 

(2) no cloud and rain fraction above 6 km occurs before 06 LST.

In [ ]:
import pandas as pd
from func import cal_buoyancy
import glob
import os
import re
import datetime
import metpy.calc as mpcalc
from metpy.units import units
from metpy.cbook import get_test_data
from metpy.plots import add_metpy_logo, SkewT
from pyhdf import SD
from scipy.interpolate import interp1d
from scipy.optimize import bisect
import numpy as np
import pickle
import numpy as np
import math
import pint
from pydoc import help
from pyhdf.SD import *
import xarray as xr
import netCDF4 as nc
import re
import matplotlib.pyplot as plt
from numpy import load
from numpy import asarray
from numpy import save
import pytz
from scipy import stats
from sklearn.linear_model import LinearRegression
from collections import Counter
import pymannkendall as mk
%matplotlib inline

In [2]:
import sys
print(sys.version)

3.9.13 (main, Oct 13 2022, 21:15:33) 
[GCC 11.2.0]


In [2]:
# Set display options to show all rows and columns in dataframe
pd.set_option('display.max_rows', None)

pd.set_option('display.max_columns', None)

np.set_printoptions(threshold=np.inf)

In [36]:
#from arscl1cloth maybe need cloud layer top height?

directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/arscl1cloth"

files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".cdf")]

filtered_files = []

for file in files:
    
    file_name = os.path.basename(file)
    
    year_month = file_name.split('.')[2]
    
    if len(year_month) == 8:
        
        year = int(year_month[0:4])
        
        month = int(year_month[4:6])
        
        if 5 <= month <= 9 and 2020 > year > 2000:
            
            filtered_files.append(file)


arscl1cloth = []

for ds in sorted(filtered_files):
    
    cloud = xr.open_dataset(ds)
    
    arscl1cloth.append(cloud)


In [39]:
#does not contain mask, possibly left out of analysis?

#from arsclcbh1cloth maybe need CloudBaseCeilometerCloth, CloudBaseMplScott, 
#CloudBaseMplCamp, CloudBaseMplCloth, CloudBaseCeilometerStd (std = standard), CloudBasePrecipitation   

# directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth"

# files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".cdf")]

# filtered_files = []

# for file in files:
    
#     file_name = os.path.basename(file)
    
#     year_month = file_name.split('.')[2]
    
#     if len(year_month) == 8:
        
#         year = int(year_month[0:4])
        
#         month = int(year_month[4:6])
        
#         if 5 <= month <= 9 and 2020 > year > 2000:
            
#             filtered_files.append(file)


# arsclcbh1cloth = []

# for ds in sorted(filtered_files):
    
#     print(ds)
    
#     cloud = xr.open_dataset(ds)
    
#     arsclcbh1cloth.append(cloud)


../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010501.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010502.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010503.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010504.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010505.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010506.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010507.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010508.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010509.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010731.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010801.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010802.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010803.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010804.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010805.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010806.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010807.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20010808.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020618.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020619.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020620.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020621.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020622.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020623.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020624.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020625.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020626.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020926.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020927.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020928.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020929.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20020930.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030501.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030502.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030503.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030504.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030811.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030812.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030813.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030814.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030815.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030816.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030817.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030818.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20030819.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

...

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080616.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080617.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080618.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080619.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080620.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080621.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080622.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080623.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080624.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080909.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080910.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080911.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080912.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080913.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080914.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080915.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080916.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20080917.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090704.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090705.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090706.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090707.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090708.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090709.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090710.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090711.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090712.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20090930.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100501.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100502.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100503.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100504.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100505.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100506.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100507.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100508.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100804.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100805.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100806.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100807.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100808.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100809.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100810.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100811.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.20100812.000000.cdf
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclcbh1cloth/sgparsclcbh1clothC1.c1.

In [52]:
directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias"

files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".nc")]

filtered_files = []

for file in files:
    
    file_name = os.path.basename(file)
    
    year_month = file_name.split('.')[2]
    
    if len(year_month) == 8:
        
        year = int(year_month[0:4])
        
        month = int(year_month[4:6])
        
        if 5 <= month <= 9 and 2020 > year > 2000:
            
            filtered_files.append(file)


arsclkazr1kollias = []

for ds in sorted(filtered_files):
    
    print(ds)
    
    cloud = xr.open_dataset(ds)
    
    arsclkazr1kollias.append(cloud)

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110501.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110502.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110503.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110504.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110505.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110506.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110507.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110508.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20110509.000000.nc
../../data/rong1/climate_fu/Data/DOEA

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130610.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130611.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130612.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130613.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130614.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130615.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130616.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130617.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130618.000000.nc
../../data/rong1/climate_fu/Data/DOEA

../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130826.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130827.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130828.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130829.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130830.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130831.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130901.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130902.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kollias/sgparsclkazr1kolliasC1.c1.20130903.000000.nc
../../data/rong1/climate_fu/Data/DOEA

In [524]:
all_vars = list(cloud14_19[0].variables)

vars_to_keep = ['base_time','time_offset','time','layer','height','lat','lon','alt','precip_mean','qc_precip_mean', 'cloud_base_best_estimate', 'cloud_layer_base_height','cloud_layer_top_height','cloud_mask_mplzwang']

vars_to_drop = list(set(all_vars) - set(vars_to_keep))

In [525]:
#2011 - 2013

directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/arsclkazr1kolliasC1_gloria"

files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".nc")]

filtered_files = []

for file in files:
    
    file_name = os.path.basename(file)
    
    year_month = file_name.split('.')[2]
    
    if len(year_month) == 8:
        
        year = int(year_month[0:4])
        
        month = int(year_month[4:6])
        
        if 5 <= month <= 9 and 2020 > year > 2000 and ".c1." in file_name:
            
            filtered_files.append(file)

            
filtered_files = sorted(filtered_files)
    

In [528]:
cloud11_13 = []

for ds in filtered_files:
    
        cloud = xr.open_dataset(ds, drop_variables=vars_to_drop)
        
        cloud11_13.append(cloud)

In [178]:
#2014-2019

directory_path = "../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0"

files = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".nc")]

filtered_files = []

for file in files:
    
    file_name = os.path.basename(file)
    
    year_month = file_name.split('.')[2]
    
    if len(year_month) == 8:
        
        year = int(year_month[0:4])
        
        month = int(year_month[4:6])
        
        if 5 <= month <= 9 and 2020 > year > 2000:
            
            filtered_files.append(file)

            
filtered_files = sorted(filtered_files)

len(filtered_files)

918

In [114]:
os.environ['HDF5_USE_FILE_LOCKING']='FALSE'

In [182]:
cloud14_19 = []

for ds in filtered_files:
    
        cloud = xr.open_dataset(ds)
        
        print(ds)
        
        cloud14_19.append(cloud)

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140501.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140502.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140503.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140504.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140505.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140506.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140507.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140508.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140712.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140713.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140714.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140715.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140716.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140717.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140718.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140719.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140922.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140923.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140924.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140925.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140926.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140927.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140928.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20140929.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150703.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150704.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150705.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150706.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150707.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150708.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150709.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150710.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150913.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150914.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150915.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150916.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150917.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150918.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150919.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20150920.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

...

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170829.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170830.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170831.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170901.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170902.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170903.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170904.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20170905.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180609.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180610.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180611.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180612.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180613.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180614.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180615.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180616.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180820.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180821.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180822.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180823.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180824.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180825.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180826.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20180827.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190606.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190607.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190608.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190609.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190610.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190611.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190612.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190613.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190817.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190818.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190819.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190820.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190821.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190822.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190823.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclkazr1kolliasC1.c0.20190824.000000.nc
../../data/rong1/climate_fu/Data/DOEARM/sgp/sgparsclkazr1kolliasC1.c0/sgparsclka

In [518]:
#target corresponds to 11:30 LST

cloud_top_heights = []

target = 15750

for df in cloud14_19:

    bt = pd.Timestamp(df.variables['base_time'].data)
    
    offset = pd.Timestamp(df.variables['time_offset'].data[target])
    
    time_final = (bt.year, bt.month, bt.day, offset.hour, offset.minute, offset.second)
    
    time_final = datetime.datetime(*time_final)
    
    height = df.variables['cloud_layer_top_height'].data[target]
    
    tup = (time_final, height)
    
    cloud_top_heights.append(tup)

cth_df = pd.DataFrame(cloud_top_heights, columns=['time','ct_heights'])

cth_df.head()

## Shallow Convection Conditions

In [519]:
# def has_consecutive_set(arr):
    
#     consecutive_count = 0

#     for i in range(len(arr) - 1):
        
#         if arr[i] + 1 == arr[i + 1]:
            
#             consecutive_count += 1
            
#             if consecutive_count >= 900:  # Check for at least 5 consecutive increasing integers
                
#                 return True
#         else:
            
#             consecutive_count = 0

#     return False

In [465]:
# #shallow: no cloud fraction above 6km between 8-18 LST (14 - 24)

# #finding index for 8 -- 18 LST

# shallows = 0

# nonshallows = 0 

# for df_index in cloud14_19:

#     cldf = df_index

#     offset = cldf['time_offset'].data

#     timestamps = pd.to_datetime(t)

#     hours_only = np.array(timestamps.hour)

#     hr_14_index = np.where(hours_only == 14)[0][0]

#     hr_24_index = np.where(hours_only == 23)[0][-1]


#     #corresponds to 6km 

#     _6km_index = 194

#     _toa_index = 596

#     _sfc_index = 0


#     cloudmask = cldf['cloud_mask_mplzwang'].data

#     _8_18lst = slice(hr_14, hr_24)

#     _6km_to_top = slice(_6km_index, _toa_index)

#     _sfc_to_6km = slice(_sfc_index, _6km_index)

#     cond1 = cloudmask[_8_18lst, _6km_to_top]

#     #if true, shallow convection condition 1) met

#     shallow_above = np.all(cond1 != 1)

#     # cloud fraction below 6km lasts for more than one hour between 8 -- 18 LST

#     cond2 = cloudmask[_8_18lst, _sfc_to_6km]

#     shallow_below = np.where(cond2 == 1)

#     check_consecutive_1hr = np.unique(shallow_below[0])

#     #if true, shallow convection condition 2) met

#     shallow_below = has_consecutive_set(check_consecutive_1hr)

#     if shallow_above == True and shallow_below == True:
        
#         shallows+=1
#     else: 
#         nonshallows+=1



In [ ]:
cloud01_10 = arscl1cloth

In [711]:
#time resolution: every 10 seconds (8640)
#vertical resolution: every 45 meters (512)
#8640 x 512 --> shape of array 

#CloudMaskMplCamp

#CloudMaskMplCloth

nanscloth_list = []

for df in cloud01_10:

    time = df['base_time'].data
    
    try:
    
        cloth = df['CloudMaskMplCloth'].data

        flatcloth = cloth.flatten()

        totalcloth = flatcloth.shape[0]

        nanscheckcloth = flatcloth[flatcloth < 0]

        nanscloth = nanscheckcloth.shape[0]

        ratiocloth = (nanscloth/totalcloth)*100
    
        if ratiocloth > 90:

            tup = (time, ratiocloth)

            nanscloth_list.append(tup)
            
    except Exception as e: 
            
        cloth = df['CloudMaskMplZwang'].data
        
        flatcloth = cloth.flatten()

        totalcloth = flatcloth.shape[0]

        nanscheckcloth = flatcloth[flatcloth < 0]

        nanscloth = nanscheckcloth.shape[0]

        ratiocloth = (nanscloth/totalcloth)*100
        
        if ratiocloth > 0:

            tup = (time, ratiocloth)

            nanscloth_list.append(tup)


In [712]:
nansdf_01_10 = pd.DataFrame(nanscloth_list, columns=['date','ratio'])

nansdf_01_10.index = nansdf_01_10['date']

len(nansdf_01_10.index)

269

In [651]:
_allcloud = cloud11_13 + cloud14_19 

In [703]:
#16650 --> 16831 = 11:30 --> 11:42 LST

#start checking nans after 11th index 

t = cloud14_19[0]['cloud_mask_mplzwang'].data[16651][11:]

np.sum(np.isnan(t))

0

In [708]:
16650+181

16831

In [557]:
#total values in cloud_mask without nans (21600 x 596) - 237600

total = 12636000

nanslist = []

for df in _allcloud:
    
    time = df['base_time'].data
    
    nans = np.isnan(df['cloud_mask_mplzwang'].data)

    count = np.sum(nans) - 237600
    
    #the first 11 values in every element of array will be nans == 237600 total values of nans to be expected
    #if more than that, add this df to a list which contains nan ratio in cloudmask
    
    if count > 0:
    
        ratio = count/total

        tup = (time, ratio)
        
        nanslist.append(tup)
    

In [710]:
nansdf = pd.DataFrame(nanslist, columns = ['time','ratio'])

nansdf.index = nansdf['time']

nansdf['ratio'] = nansdf['ratio']*100

nansdf = nansdf[nansdf['ratio'] > 0]

nansdf.shape

(114, 2)

In [713]:
#to filter out in fig 2&3

cloudmasknans = list(nansdf.index) + list(nansdf_01_10.index)

print(len(cloudmasknans))

with open('cloudmasknans.pkl', 'wb') as file:
    
    pickle.dump(cloudmasknans, file)


383
